In [110]:
import requests

def get_exchange_info():
    url = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

exchange_info = get_exchange_info()


In [111]:
def get_tradable_futures_pairs(exchange_info):
    if exchange_info and 'symbols' in exchange_info:
        futures_pairs = [pair["symbol"] for pair in exchange_info['symbols'] if pair['quoteAsset'] == 'USDT' and pair['isMarginTradingAllowed']] 
        return futures_pairs
    else:
        return None

In [112]:
 tradable_pairs = get_tradable_futures_pairs(exchange_info)

In [108]:
def convert_pair_name(pair):
    usdt_index = pair.find("USDT")
    if usdt_index != -1:
        base_asset = pair[:usdt_index]
        quote_asset = pair[usdt_index:]
        return f"{base_asset}_{quote_asset}"
    else:
        return pair


In [57]:
import os
import requests

import os.path

def download_feather_files(tradable_pairs):
    base_url = "https://github.com/DigiTuccar/HistoricalDataForTradeBacktest/raw/main/binance"
    save_directory = "../compressed"  # Directory to save the files
    
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    for pair in tradable_pairs:
        formatted_pair = convert_pair_name(pair)
        file_url = f"{base_url}/{formatted_pair}-15m.feather"  # Assuming the interval is 15 minutes (15m)
        file_path = os.path.join(save_directory, f"{formatted_pair}-15m.feather")
        
        if os.path.exists(file_path):
            print(f"File {formatted_pair}-15m.feather already exists. Skipping...")
            continue
        
        response = requests.get(file_url)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded {formatted_pair}-15m.feather")
        else:
            print(f"Failed to download {formatted_pair}-15m.feather")





In [59]:
#download_feather_files(tradable_pairs)


In [66]:
import requests

# URL for the coinInfo page of the "AI" coin
url = "https://www.binance.com/en/markets"
response = requests.get(url)

if response.status_code == 200:
    # Save the HTML content to a file
    with open("coinInfo.html", "w", encoding="utf-8") as f:
        f.write(response.text)
    print("HTML content saved to coinInfo.html")

    # Now you can parse the HTML content using BeautifulSoup
    # Add your BeautifulSoup parsing code here
else:
    print("Failed to retrieve the webpage")


HTML content saved to coinInfo.html


In [60]:
import requests
from bs4 import BeautifulSoup

url = "https://www.binance.com/en/markets/"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the section that contains market information
    market_section = soup.find('section', class_='sc-4ymgbe-2 dXJslv')
    if market_section:
        # Extract market names and other relevant information
        markets = market_section.find_all('div', class_='sc-1eb5slv-0 gwFoVT')
        for market in markets:
            market_name = market.find('div', class_='sc-1eb5slv-1 diHoyv').text.strip()
            market_info = market.find('div', class_='sc-1eb5slv-2 cePoPx').text.strip()
            print(f"Market: {market_name}, Info: {market_info}")
else:
    print("Failed to retrieve the webpage")


In [67]:
start_div = soup.find('div', id='__APP_TOP_PORTAL').find_next_sibling('div', id='__APP_HEADER')


In [75]:
#start_div

In [72]:
#soup

In [73]:
start_point = soup.find('script', id='__APP_DATA')

In [78]:
#start_point

In [82]:
if start_point:
    # Find the next <script> tag after the starting point
    end_script_tag = start_point.find_next('script')
    
    # Extract the HTML content between the starting point and the end script tag
    extracted_html = str(start_point) + str(end_script_tag.previous_sibling)
    

In [86]:
import json
# Find the script tag with id="__APP_DATA" and type="application/json"
script_tag = soup.find('script', id='__APP_DATA', type='application/json')

if script_tag:
    # Get the JSON content from the script tag
    json_content = script_tag.string
    
    # Parse the JSON content
    try:
        json_data = json.loads(json_content)
        print("JSON data after script tag:")
        print(json_data)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
else:
    print("Script tag not found")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [97]:
json_data['pageData']['redux'].keys()

dict_keys(['markets', 'global', 'cookies', 'products', 'ssrStore', 'userCenter', 'persistSetting'])

In [95]:
json_data['pageData']['redux']["markets"].keys()

dict_keys(['tabInfo', 'search', 'showTradingDataInfo'])

In [98]:
json_data['pageData']['redux']['global'].keys()

dict_keys(['origin', 'url', 'path', 'host', 'dir', 'metaData', 'region', 'isHybrid', 'withChat', 'withHeader', 'withFooter', 'withCompliance', 'restrictedBusinessKeys', 'noticeInfo', 'pageLayout', 'isMobile'])

In [103]:
len(json_data['pageData']['redux']['products']["productMap"].keys())

1168

In [104]:
json_data['pageData']['redux']['products']["productMap"]

{'low': '51677.00',
 'close': '51737.82',
 'high': '52488.77',
 'open': '52197.11',
 'volume': '29362.47',
 'quoteVolume': '1530420724.3326896',
 'symbol': 'BTCUSDT',
 'tickSize': '0.01',
 'minQty': '0.00001000',
 'quoteAsset': 'USDT',
 'baseAsset': 'BTC',
 'quoteName': 'TetherUS',
 'baseAssetName': 'Bitcoin',
 'parentMarket': 'USDT',
 'parentMarketName': 'USDT',
 'circulatingSupply': 19631775,
 'tags': ['pow', 'mining-zone']}

In [ ]:
from bs4 import BeautifulSoup
import json
import requests

def extract_json_from_html(url):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find the script tag with id="__APP_DATA" and type="application/json"
            script_tag = soup.find('script', id='__APP_DATA', type='application/json')
            
            if script_tag:
                # Get the JSON content from the script tag
                json_content = script_tag.string
                
                # Parse the JSON content
                json_data = json.loads(json_content)
                
                # Access the desired information
                products = json_data['pageData']['redux']['products']['productMap']
                
                # Return the products information
                return products
            else:
                return None, "Script tag not found"
        else:
            return None, f"Failed to retrieve the webpage. Status code: {response.status_code}"
    except Exception as e:
        return None, f"An error occurred: {str(e)}"

# Example usage:
url = "https://www.example.com"  # Replace with your URL
products_info, error_message = extract_json_from_html(url)
if products_info:
    print("Products Information:")
    print(products_info)
else:
    print(f"Error: {error_message}")


In [117]:
coin_infos = json_data['pageData']['redux']['products']["productMap"]


In [133]:
filter_coin_infos = [] 
for key,value in coin_infos.items():
    if key.endswith("USDT"):
        filter_coin_infos.append({
            "symbol":key,
            'baseAssetName':value['baseAssetName'],
            "tags":value["tags"]
        })

In [134]:
import pandas as pd
df = pd.DataFrame(filter_coin_infos)


In [135]:
df.head()

,symbol,baseAssetName,tags
0,BTCUSDT,Bitcoin,"[pow, mining-zone]"
1,1INCHUSDT,1inch,[defi]
2,AAVEUSDT,Aave,[defi]
3,ACMUSDT,AC Milan Fan Token,[fan_token]
4,ADAUSDT,Cardano,"[Layer1_Layer2, pos, mining-zone]"


In [136]:
df.to_csv("coin_tags.csv")

In [132]:
coin_infos["BTCUSDT"]

{'low': '51677.00',
 'close': '51737.82',
 'high': '52488.77',
 'open': '52197.11',
 'volume': '29362.47',
 'quoteVolume': '1530420724.3326896',
 'symbol': 'BTCUSDT',
 'tickSize': '0.01',
 'minQty': '0.00001000',
 'quoteAsset': 'USDT',
 'baseAsset': 'BTC',
 'quoteName': 'TetherUS',
 'baseAssetName': 'Bitcoin',
 'parentMarket': 'USDT',
 'parentMarketName': 'USDT',
 'circulatingSupply': 19631775,
 'tags': ['pow', 'mining-zone']}